In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

import os 
from glob import glob 

from PIL import Image, ImageOps
import albumentations as A
import cv2

from tqdm import tqdm 

from pathlib import Path

In [7]:
df = pd.read_csv('./data/train.csv')
display(df)

,ID,target
0,002f99746285dfdd.jpg,16
1,008ccd231e1fea5d.jpg,10
2,008f5911bfda7695.jpg,10
3,009235e4c9c07af5.jpg,4
4,00b2f44967580c74.jpg,16
...,...,...
1565,fed9e9ec4a77bc06.jpg,4
1566,feeade617aa68c45.jpg,7
1567,ff51dd281a8423f1.jpg,11
1568,ff8a6a251ce51c95.jpg,5


In [86]:
data_path = './data/'
train_path = os.path.join(data_path, "train")  # 원본 이미지 폴더

# 'augmented/' 폴더가 없으면 생성
augmented_folder_path = os.path.join(data_path, "augmented")
os.makedirs(augmented_folder_path, exist_ok=True)

# 'augmented/rotate' 폴더가 없으면 생성
rotate_path = os.path.join(augmented_folder_path, "rotate")
os.makedirs(rotate_path, exist_ok=True)

ids = []
targets = []

for index, row in tqdm(df.iterrows(), desc='Image rotation', total=df.shape[0]):
    ID = row['ID']  # 이미지 파일명 or ID
    target = row['target']  # 대상 클래스 or 레이블
    image_path = os.path.join(train_path, ID)  # 원본 이미지 경로

    # 90도 회전
    id_90 = 'r90_' + ID
    ids.append(id_90)
    targets.append(target)
    rotate90_image = np.array(ImageOps.exif_transpose(Image.open(image_path).rotate(90, expand=True)))
    Image.fromarray(rotate90_image).save(os.path.join(rotate_path, id_90))
    
    # 180도 회전
    id_180 = 'r180_' + ID
    ids.append(id_180)
    targets.append(target)
    rotate180_image = np.array(ImageOps.exif_transpose(Image.open(image_path).rotate(180, expand=True)))
    Image.fromarray(rotate180_image).save(os.path.join(rotate_path, id_180))

    # 270도 회전
    id_270 = 'r270_' + ID
    ids.append(id_270)
    targets.append(target)
    rotate270_image = np.array(ImageOps.exif_transpose(Image.open(image_path).rotate(270, expand=True)))
    Image.fromarray(rotate270_image).save(os.path.join(rotate_path, id_270))

# 증강된 이미지 데이터를 DataFrame에 추가하고 결과 저장
rotate_data = {'ID': ids, 'target': targets}
rotate_df = pd.DataFrame(rotate_data)

Image rotation: 100%|██████████| 1570/1570 [00:38<00:00, 40.59it/s]


In [87]:
# 원본 데이터셋의 크기
original_size = len(df)
# 증강된 데이터셋의 크기
rotate_size = len(rotate_df)
# 전체 데이터셋의 크기
total_size = len(df) + len(rotate_df)

print(f"원본 데이터셋 크기: {original_size}")
print(f"rotate 증강된 데이터셋 크기: {rotate_size}")
print(f"전체 데이터셋 크기: {total_size}")

원본 데이터셋 크기: 1570
rotate 증강된 데이터셋 크기: 4710
전체 데이터셋 크기: 6280


In [88]:
rotate_files = os.listdir(rotate_path)
print(f"'rotate' 폴더 내의 파일 수: {len(rotate_files)}")

'rotate' 폴더 내의 파일 수: 4710


In [89]:
rotate_df.to_csv('./data/rotate_train.csv', index=False)

In [8]:
rotate_df = pd.read_csv('./data/rotate_train.csv')
display(rotate_df)

,ID,target
0,r90_002f99746285dfdd.jpg,16
1,r180_002f99746285dfdd.jpg,16
2,r270_002f99746285dfdd.jpg,16
3,r90_008ccd231e1fea5d.jpg,10
4,r180_008ccd231e1fea5d.jpg,10
...,...,...
4705,r180_ff8a6a251ce51c95.jpg,5
4706,r270_ff8a6a251ce51c95.jpg,5
4707,r90_ffc22136f958deb1.jpg,9
4708,r180_ffc22136f958deb1.jpg,9


In [92]:
# 앞서 정의한 변환들
H_flip = A.Compose([A.HorizontalFlip(always_apply=True)])
V_flip = A.Compose([A.VerticalFlip(always_apply=True)])

data_path = './data/'

# 'augmented/' 폴더가 없으면 생성
augmented_folder_path = os.path.join(data_path, "augmented")
os.makedirs(augmented_folder_path, exist_ok=True)

# 'augmented/rotateflip' 폴더가 없으면 생성
rotateflip_path = os.path.join(augmented_folder_path, "rotateflip")
os.makedirs(rotateflip_path, exist_ok=True)

# 'augmented/rotate' 폴더가 없으면 생성
rotate_path = os.path.join(augmented_folder_path, "rotate")
os.makedirs(rotate_path, exist_ok=True)

# 새로운 데이터프레임을 위한 리스트
new_filenames = []  # 수정된 부분: 변수 이름을 new_filename에서 new_filenames로 변경
new_targets = []

# rotate_df에 있는 모든 항목에 대해 반복 (rotate_df는 미리 정의되어 있어야 합니다)
for _, row in tqdm(rotate_df.iterrows(), desc='Processing images', total=rotate_df.shape[0]):
    original_id = row['ID']
    target = row['target']
    
    # 원본 이미지 파일 경로
    image_path = os.path.join(rotate_path, original_id)  # 파일 확장자가 이미 ID에 포함되어 있다고 가정
    
    if os.path.exists(image_path):
        image = np.array(Image.open(image_path))

        # 파일명에 따라 수행할 변환 결정 및 적용
        if original_id.startswith('r90_') or original_id.startswith('r270_'):
            flipped_image = V_flip(image=image)['image']
            new_id = 'Vflip_' + original_id  # new_id 변수 이름 변경
        else:
            flipped_image = H_flip(image=image)['image']
            new_id = 'Hflip_' + original_id  # new_id 변수 이름 변경

        # 새 파일명으로 이미지 저장
        Image.fromarray(flipped_image).save(os.path.join(rotateflip_path, new_id))

        # 리스트에 정보 추가
        new_filenames.append(new_id)  # 수정된 부분: 올바른 리스트 변수 사용
        new_targets.append(target)

# 새로운 데이터프레임 생성 및 저장
rotateflip_df = pd.DataFrame({
    'ID': new_filenames,
    'target': new_targets
})


Processing images: 100%|██████████| 4710/4710 [00:33<00:00, 139.75it/s]


In [93]:
# 원본 데이터셋의 크기
original_size = len(df)
# 증강된 데이터셋의 크기
rotate_size = len(rotate_df)
rotateflip_size = len(rotateflip_df)
# 전체 데이터셋의 크기
total_size = len(df) + len(rotate_df) + len(rotateflip_df)

print(f"원본 데이터셋 크기: {original_size}")
print(f"rotate 증강된 데이터셋 크기: {rotate_size}")
print(f"rotateflip 증강된 데이터셋 크기: {rotateflip_size}")
print(f"전체 데이터셋 크기: {total_size}")

원본 데이터셋 크기: 1570
rotate 증강된 데이터셋 크기: 4710
rotateflip 증강된 데이터셋 크기: 4710
전체 데이터셋 크기: 10990


In [94]:
rotateflip_path_files = os.listdir(rotateflip_path)
print(f"'rotateflip' 폴더 내의 파일 수: {len(rotateflip_path_files)}")

'rotateflip' 폴더 내의 파일 수: 4710


In [95]:
rotateflip_df.to_csv('./data/rotateflip_train.csv', index=False)

In [9]:
rotateflip_df = pd.read_csv('./data/rotateflip_train.csv')
display(rotateflip_df)

,ID,target
0,Vflip_r90_002f99746285dfdd.jpg,16
1,Hflip_r180_002f99746285dfdd.jpg,16
2,Vflip_r270_002f99746285dfdd.jpg,16
3,Vflip_r90_008ccd231e1fea5d.jpg,10
4,Hflip_r180_008ccd231e1fea5d.jpg,10
...,...,...
4705,Hflip_r180_ff8a6a251ce51c95.jpg,5
4706,Vflip_r270_ff8a6a251ce51c95.jpg,5
4707,Vflip_r90_ffc22136f958deb1.jpg,9
4708,Hflip_r180_ffc22136f958deb1.jpg,9


In [15]:
transforms = A.Compose([
    A.OneOf([
        # 그리드 왜곡: 이미지에 그리드를 적용하고 그리드 포인트를 왜곡합니다. num_steps=5는 그리드의 세분성을 결정합니다.
        A.GridDistortion(num_steps=5, distort_limit=0.3, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=True, p=1),
        # 탄성 변형: 이미지에 탄성적 왜곡을 추가하여 자연스러운 변형을 시뮬레이션합니다.
        A.ElasticTransform(always_apply=True, p=1, alpha=1.0, sigma=50.0, alpha_affine=50.0, interpolation=0, border_mode=1, value=(0, 0, 0), mask_value=None, approximate=False),
        # 광학 왜곡: 이미지에 광학적 왜곡 효과를 추가합니다. distort_limit=(-0.3, -0.1)은 왜곡의 범위를 정합니다.
        A.OpticalDistortion(always_apply=True, p=1, distort_limit=(-0.3, -0.1)),
        # 광학 왜곡: distort_limit=(0.1, 0.3)으로 설정하여 왜곡의 정도와 방향을 변형합니다.
        A.OpticalDistortion(always_apply=True, p=1, distort_limit=(0.1, 0.3)),
    ], p=0.85),
    A.SomeOf([
        # 밝기 및 대비 조정: 이미지의 밝기와 대비를 무작위로 조절합니다.
        A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=1),
        # 색조, 채도, 밝기 조정: 이미지의 색조(Hue), 채도(Saturation), 밝기(Value)를 무작위로 변형합니다.
        A.HueSaturationValue(hue_shift_limit=15, sat_shift_limit=25, val_shift_limit=20, p=1),
        # 곱셈 노이즈: 이미지에 곱셈 노이즈를 추가하여 픽셀 값에 변화를 줍니다.
        A.MultiplicativeNoise(p=1, multiplier=(1, 1.5), per_channel=True),
        # 히스토그램 평활화: 이미지의 대비를 향상시키기 위해 히스토그램 평활화를 적용합니다.
        A.Equalize(p=1, mode='cv', by_channels=True),
    ], n=2, p=0.85),
    A.OneOf([
        # 회전: 지정된 각도 범위 내에서 이미지를 회전합니다.
        A.Rotate(limit=(10, 30), border_mode=cv2.BORDER_CONSTANT, p=1),
        A.Rotate(limit=(150, 170), border_mode=cv2.BORDER_CONSTANT, p=1),
        A.Rotate(limit=(190, 210), border_mode=cv2.BORDER_CONSTANT, p=1),
        A.Rotate(limit=(330, 350), border_mode=cv2.BORDER_CONSTANT, p=1),
    ], p=1),
    # 드롭아웃: 이미지에서 무작위 영역을 제거하여 드롭아웃 효과를 만듭니다.
    A.CoarseDropout(p=0.5, max_holes=40, max_height=15, max_width=15, min_holes=8, min_height=8, min_width=8),
    # 히스토그램 평활화: 이미지의 대비를 개선합니다.
    A.Equalize(p=0.5, mode='cv', by_channels=True),
    A.OneOf([
        # 블러: 이미지를 흐리게 합니다.
        A.Blur(blur_limit=(3, 4), p=1),
        # 모션 블러: 움직임이 있는 것처럼 이미지에 블러 효과를 추가합니다.
        A.MotionBlur(blur_limit=(3, 5), p=1),
        # 다운스케일: 이미지의 해상도를 낮춥니다. 이는 이미지의 품질을 인위적으로 감소시킵니다.
        A.Downscale(scale_min=0.455, scale_max=0.5, interpolation=2, p=1),
    ], p=0.5),
    # 가우스 노이즈: 이미지에 가우시안 노이즈를 추가합니다. 이는 이미지에 무작위 노이즈를 삽입하여 모델이 노이즈에 강인하게 만듭니다.
    A.GaussNoise(var_limit=(100, 800), per_channel=True, p=0.5),
])


In [16]:
# CSV 파일들을 하나의 데이터 프레임으로 결합
combined_df = pd.concat([df, rotate_df, rotateflip_df], ignore_index=True)

# 원본 및 회전/뒤집기된 이미지가 있는 경로
data_paths = ['./data/train/', './data/augmented/rotate/', './data/augmented/rotateflip/']

# 최종 증강된 이미지를 저장할 경로
augmented_save_path = './data/augmented/A_rotateandflip/'
os.makedirs(augmented_save_path, exist_ok=True)  # 폴더가 없으면 생성

ids = []
targets = []

for data_path in data_paths:
    image_files = [f for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f))]
    for image_file in tqdm(image_files, desc=f'Processing {data_path}'):
        # 데이터 프레임에서 해당 이미지 파일의 target 값을 찾음
        target_row = combined_df[combined_df['ID'] == image_file]
        if not target_row.empty:
            target = target_row['target'].iloc[0]
            # 이미지 증강 및 저장 과정
            image_path = os.path.join(data_path, image_file)
            image = np.array(Image.open(image_path))
            transformed_image = transforms(image=image)['image']
            save_path = os.path.join(augmented_save_path, image_file)  # 파일 확장자를 유지
            Image.fromarray(transformed_image).save(save_path)
            ids.append(image_file)
            targets.append(target)

# 증강된 이미지 데이터를 DataFrame에 추가하고 결과 저장
aug_df = pd.DataFrame({'ID': ids, 'target': targets})


Processing ./data/train/:   0%|          | 0/1570 [00:00<?, ?it/s]

Processing ./data/augmented/rotateflip/: 100%|██████████| 4710/4710 [04:35<00:00, 17.08it/s]


In [17]:
# 원본 데이터셋의 크기
original_size = len(df)
# 증강된 데이터셋의 크기
rotate_size = len(rotate_df)
rotateflip_size = len(rotateflip_df)
augmented_size = len(aug_df)
# 전체 데이터셋의 크기
total_size = len(df) + len(rotate_df) + len(rotateflip_df) + len(aug_df)

print(f"원본 데이터셋 크기: {original_size}")
print(f"rotate 증강된 데이터셋 크기: {rotate_size}")
print(f"rotateflip 증강된 데이터셋 크기: {rotateflip_size}")
print(f"증강완료된 데이터셋 크기: {augmented_size}")
print(f"전체 데이터셋 크기: {total_size}")

원본 데이터셋 크기: 1570
rotate 증강된 데이터셋 크기: 4710
rotateflip 증강된 데이터셋 크기: 4710
증강완료된 데이터셋 크기: 10990
전체 데이터셋 크기: 21980


In [18]:
augmented_files = os.listdir(augmented_save_path)
print(f"'A_rotateandflip' 폴더 내의 파일 수: {len(augmented_files)}")

'A_rotateandflip' 폴더 내의 파일 수: 10990


In [101]:
aug_df.to_csv('./data/A_rotateandflip_train.csv', index=False)

In [1]:
# import shutil
# import os

# # 증강된 이미지가 저장된 폴더 삭제
# rotate_path = './data/augmented/rotate'
# if os.path.exists(rotate_path):
#     shutil.rmtree(rotate_path)
#     print(f"'{rotate_path}' 폴더와 그 안의 모든 이미지가 삭제되었습니다.")
    
# # 증강된 이미지가 저장된 폴더 삭제
# rotateflip_path = './data/augmented/rotateflip'
# if os.path.exists(rotateflip_path):
#     shutil.rmtree(rotateflip_path)
#     print(f"'{rotateflip_path}' 폴더와 그 안의 모든 이미지가 삭제되었습니다.")


# albumentations_path = './data/augmented/A_rotateandflip'
# if os.path.exists(albumentations_path):
#     shutil.rmtree(albumentations_path)
#     print(f"'{albumentations_path}' 폴더와 그 안의 모든 이미지가 삭제되었습니다.")


# # 생성된 CSV 파일 삭제
# csv_file_path = './data/rotate_train.csv'
# if os.path.exists(csv_file_path):
#     os.remove(csv_file_path)
#     print(f"'{csv_file_path}' 파일이 삭제되었습니다.")
    
# csv_file_path = './data/rotateflip_train.csv'
# if os.path.exists(csv_file_path):
#     os.remove(csv_file_path)
#     print(f"'{csv_file_path}' 파일이 삭제되었습니다.")
    
# csv_file_path = './data/A_rotateandflip_train.csv'
# if os.path.exists(csv_file_path):
#     os.remove(csv_file_path)
#     print(f"'{csv_file_path}' 파일이 삭제되었습니다.")

'./data/augmented/A_rotateandflip' 폴더와 그 안의 모든 이미지가 삭제되었습니다.
'./data/A_rotateandflip_train.csv' 파일이 삭제되었습니다.
